In [1]:
%pip install --upgrade pip
%pip install pandas
%pip install lxml
%pip install html5lib
%pip install beautifulsoup4
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
import numpy as np
import re

In [3]:
espn_standings = pd.read_html("https://www.espn.com/soccer/standings/_/league/esp.1")

# Extract the two DataFrames
teams_df = espn_standings[0]
stats_df = espn_standings[1]

# Combine them into a single DataFrame by concatenating along the columns (axis=1)
standings = pd.concat([teams_df, stats_df], axis=1)

# Remove the position numbers and team abbreviations, drop the original Team column
standings['Team'] = standings[standings.columns[0]].str.extract(r'\d{1}[A-Z]{3}(\D+)', expand=False).str.strip()
standings.drop(standings.columns[0], axis=1, inplace=True)

# Reset the index, reorder the columns
standings.reset_index(drop=True, inplace=True)
standings = standings[["Team","GP","W","D","L","F","A","GD","P"]]
standings.index = np.arange(1, len(standings)+1)

display(standings)

,Team,GP,W,D,L,F,A,GD,P
1,Barcelona,8,7,0,1,25,9,16,21
2,Real Madrid,8,5,3,0,17,6,11,18
3,Villarreal,8,5,2,1,17,15,2,17
4,Atlético Madrid,8,4,4,0,12,4,8,16
5,Athletic Club,8,4,2,2,12,8,4,14
6,Mallorca,8,4,2,2,8,6,2,14
7,Osasuna,8,4,2,2,12,13,-1,14
8,Real Betis,8,3,3,2,8,7,1,12
9,Rayo Vallecano,8,2,4,2,9,8,1,10
10,Celta Vigo,8,3,1,4,15,15,0,10


In [4]:
fbref_la_liga = pd.read_html("https://fbref.com/en/comps/12/2023-2024/2023-2024-La-Liga-Stats")

# Extract only the first table containing historical data on La Liga, filter out teams that moved to Segunda Division
la_liga_historical = fbref_la_liga[0]
la_liga_historical = la_liga_historical[~la_liga_historical['Notes'].str.contains("Relegated", na=False)]
la_liga_historical.index = np.arange(1, len(la_liga_historical)+1)

display(la_liga_historical)


,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
1,1,Real Madrid,38,29,8,1,87,26,61,95,2.50,68.8,35.4,33.4,0.88,72061,Jude Bellingham - 19,Andriy Lunin,→ Champions League via league finish
2,2,Barcelona,38,26,7,5,79,44,35,85,2.24,77.6,41.6,35.9,0.94,39846,Robert Lewandowski - 19,Marc-André ter Stegen,→ Champions League via league finish
3,3,Girona,38,25,6,7,85,46,39,81,2.13,70.8,54.3,16.5,0.43,12520,Artem Dovbyk - 24,Paulo Gazzaniga,→ Champions League via league finish
4,4,Atlético Madrid,38,24,4,10,70,43,27,76,2.00,62.4,39.5,23.0,0.60,59121,Antoine Griezmann - 16,Jan Oblak,→ Champions League via league finish
5,5,Athletic Club,38,19,11,8,61,37,24,68,1.79,52.7,41.3,11.4,0.30,46112,Gorka Guruzeta - 14,Unai Simón,→ Europa League via cup win
6,6,Real Sociedad,38,16,12,10,51,39,12,60,1.58,44.9,41.3,3.6,0.10,31710,Mikel Oyarzabal - 9,Álex Remiro,→ Europa League via league finish
7,7,Betis,38,14,15,9,48,45,3,57,1.50,43.9,50.2,-6.3,-0.17,51259,Willian José - 10,Rui Silva,→ Europa Conference League via league finish
8,8,Villarreal,38,14,11,13,65,65,0,53,1.39,55.6,58.7,-3.2,-0.08,17957,Alexander Sørloth - 23,Filip Jørgensen,NaN
9,9,Valencia,38,13,10,15,40,45,-5,49,1.29,43.1,47.6,-4.5,-0.12,43420,Hugo Duro - 13,Giorgi Mamardashvili,NaN
10,10,Alavés,38,12,10,16,36,46,-10,46,1.21,46.2,44.7,1.5,0.04,17391,Samu Omorodion - 8,Antonio Sivera,NaN


In [5]:
fbref_segunda_div = pd.read_html("https://fbref.com/en/comps/17/2023-2024/2023-2024-Segunda-Division-Stats")

# Extract only the first table containing historical data on Segunda Division, filter for teams that moved to La Liga
segunda_div_historical = fbref_segunda_div[0]
segunda_div_historical = segunda_div_historical[segunda_div_historical['Notes'].str.contains("Promoted", na=False)]
segunda_div_historical.index = np.arange(1, len(segunda_div_historical)+1)

display(segunda_div_historical)

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
1,1,Leganés,42,20,14,8,56,27,29,74,1.76,47.4,35.6,11.8,0.28,8659,Miguel - 13,Diego Conde,Promoted
2,2,Valladolid,42,21,9,12,51,36,15,72,1.71,46.3,38.2,8.0,0.19,17826,Mamadou Sylla - 8,Jordi Masip,Promoted
3,4,Espanyol,42,17,18,7,59,40,19,69,1.64,55.0,39.9,15.1,0.36,22180,Martin Braithwaite - 22,Fernando Pacheco,"Promoted, → Semi-finals"


In [10]:
# get schedule in tabular format
espn_schedule = pd.read_html("https://www.espn.com/soccer/schedule/_/league/esp.1")

schedule = pd.concat([match for match in espn_schedule], axis=0)

# Split the 'MATCH.1' column into 'Team 2' and 'Score' columns
schedule[["Score", "Team 2"]] = schedule["MATCH.1"].str.split(" ", n=1, expand=True)
schedule['Team 1'] = schedule["MATCH"]
schedule['Location'] = schedule["location"]

# Reorder columns
#standings.reset_index(drop=True, inplace=True)
schedule = schedule[['Team 1', 'Team 2', 'Location', 'ODDS BY']]
schedule = schedule[schedule["ODDS BY"].notnull()]
schedule.index = np.arange(1, len(schedule)+1)

display(schedule)

,Team 1,Team 2,Location,ODDS BY
1,Leganés,Valencia,"Butarque, Leganés, Madrid, Spain",Line: LEG +175O/U: 1.5
2,Espanyol,Mallorca,"Stage Front Stadium, Barcelona, Spain",Line: MLL +150O/U: 1.5
3,Getafe,Osasuna,"Estadio Coliseum, Getafe, Spain",Line: GET +120O/U: 1.5
4,Las Palmas,Celta Vigo,"Estadio Gran Canaria, Las Palmas de Gran Canar...",Line: CEL +125O/U: 2.5
5,Real Valladolid,Rayo Vallecano,"José Zorrilla, Valladolid, Spain",Line: RAY +150O/U: 1.5
6,Real Madrid,Villarreal,"Santiago Bernabéu, Madrid, Spain",Line: RMA -380O/U: 3.5
7,Girona,Athletic Club,"Municipal de Montilivi, Girona, Spain",Line: GIR +135O/U: 2.5
8,Alavés,Barcelona,"Mendizorroza, Vitoria-Gasteiz, Spain",Line: BAR -180O/U: 2.5
9,Sevilla,Real Betis,"Ramón Sánchez-Pizjuán, Sevilla, Spain",Line: SEV +145O/U: 2.5
10,Real Sociedad,Atlético Madrid,"Reale Arena, San Sebastián, Spain",Line: ATM +125O/U: 2.5


In [79]:
def calculate_odds(team1, team2, market_odds):
    # Function to extract odds from the string
    def extract_odds(odds_str):
        # Use regex to find a number with optional + or - sign
        match = re.search(r'[+-]?\d+', odds_str)
        if match:
            return float(match.group())
        return None

    # Example betting odds string
    market_odds = market_odds[6:].split('/')[0]
    market_odds = extract_odds(market_odds)

    # Get standings data
    team1_standing = standings[standings["Team"] == team1]
    team2_standing = standings[standings["Team"] == team2]
    team1_points = team1_standing["P"].values[0]
    team2_points = team2_standing["P"].values[0]

    # Win probability based on standings
    team1_standing_prob = team1_points / (team1_points + team2_points)
    team2_standing_prob = team2_points / (team1_points + team2_points)

    def get_team_history(team_name):
        team_name_cleaned = team_name.replace("Real ", "").replace("Rayo ", "").strip()

        # Search in La Liga historical data for team name containment
        team_hist = la_liga_historical[la_liga_historical["Squad"].str.contains(team_name_cleaned, case=False, na=False)]
        
        # If not found, search in Segunda Division historical data
        if team_hist.empty:
            team_hist = segunda_div_historical[segunda_div_historical["Squad"].str.contains(team_name_cleaned, case=False, na=False)]
        
        # Return the DataFrame if found, otherwise return None
        return team_hist if not team_hist.empty else None


    # Retrieve historical data for both teams
    team1_hist = get_team_history(team1)
    team2_hist = get_team_history(team2)

    # Ensure both teams' historical data is found before proceeding
    if team1_hist is not None and team2_hist is not None:
        team1_hist_points = team1_hist["Pts"].values[0]
        team2_hist_points = team2_hist["Pts"].values[0]
    else:
        print(f"Historical data not found for {team1} or {team2}")

    # Win probability based on historical data
    team1_hist_prob = team1_hist_points / (team1_hist_points + team2_hist_points)
    team2_hist_prob = team2_hist_points / (team1_hist_points + team2_hist_points)

    # Convert betting odds to probability (fractional odds assumed)
    market_odds_frac = float(market_odds)  
    team1_odds_prob = 1 / (market_odds_frac + 1)
    team2_odds_prob = 1 - team1_odds_prob


    # Draw probability from current standings (based on number of draws)
    team1_draw_rate = team1_standing["D"].values[0] / team1_standing["GP"].values[0]
    team2_draw_rate = team2_standing["D"].values[0] / team2_standing["GP"].values[0]
    draw_standing_prob = (team1_draw_rate + team2_draw_rate) / 2  # Average draw rate


    # Draw probability from historical data
    team1_hist_draw_rate = team1_hist["D"].values[0] / team1_hist["MP"].values[0]
    team2_hist_draw_rate = team2_hist["D"].values[0] / team2_hist["MP"].values[0]
    draw_hist_prob = (team1_hist_draw_rate + team2_hist_draw_rate) / 2  # Average historical draw rate

    # Adjusted weights for the new probabilities
    w_standings = 0.35
    w_historical = 0.25
    w_odds = 0.2
    w_draw = 0.2  # Adding weight for draw probability

    # Win probabilities
    team1_combined_prob = (team1_standing_prob * w_standings) + (team1_hist_prob * w_historical) + (team1_odds_prob * w_odds)
    team2_combined_prob = (team2_standing_prob * w_standings) + (team2_hist_prob * w_historical) + (team2_odds_prob * w_odds)

    def calculate_draw_prob(team1_prob, team2_prob, w_draw):
        # Scaling factor for the draw based on the closeness of the teams
        closeness_factor = 1 - abs(team1_prob - team2_prob)  # Higher closeness means higher chance of draw
        w_draw = 0.2  # Base probability of a draw (can be adjusted)
        
        # Final draw probability increases when teams are closer in strength
        draw_prob = w_draw * closeness_factor
        
        # Ensure draw probability doesn't exceed the remaining total probability
        return min(draw_prob, 1 - (team1_prob + team2_prob))

    # Use this function to calculate draw probability
    draw_odds_prob = calculate_draw_prob(team1_combined_prob, team2_combined_prob, w_draw)


    # Draw probability (weighted)
    combined_draw_prob = (draw_standing_prob * w_standings) + (draw_hist_prob * w_historical) + (draw_odds_prob * w_odds)

    # Ensure probabilities sum to 1
    total_prob = team1_combined_prob + team2_combined_prob + combined_draw_prob
    team1_combined_prob /= total_prob
    team2_combined_prob /= total_prob
    combined_draw_prob /= total_prob

    # Return probabilities
    return [team1_combined_prob, team2_combined_prob, combined_draw_prob]

In [93]:
# Get the next game (first row) from the schedule dataframe
next_game = schedule.iloc[0].copy()

next_game["Team 1 Win Probability"] = 0
next_game["Team 2 Win Probability"] = 0
next_game["Draw Probability"] = 0

# Extract relevant data
team1 = next_game["Team 1"]
team2 = next_game["Team 2"]
espn_odds = next_game["ODDS BY"]

if pd.isna(espn_odds):
    print(f"Skipping calculation for {team1} vs {team2} due to missing odds.")
else:
    # Calculate win probabilities
    [t1_win, t2_win, draw] = calculate_odds(team1, team2, espn_odds)

    # Assign the calculated probabilities back to the 'schedule' dataframe
    next_game["Team 1 Win Probability"] = t1_win
    next_game["Team 2 Win Probability"] = t2_win
    next_game["Draw Probability"] = draw

next_game = pd.DataFrame([next_game])

# Display the updated next game with probabilities
display(next_game)


,Team 1,Team 2,Location,ODDS BY,Team 1 Win Probability,Team 2 Win Probability,Draw Probability
1,Leganés,Valencia,"Butarque, Leganés, Madrid, Spain",Line: LEG +175O/U: 1.5,0.341284,0.426273,0.232443


In [86]:
# Define new columns for win and draw probabilities
schedule["Team 1 Win Probability"] = 0.0
schedule["Team 2 Win Probability"] = 0.0
schedule["Draw Probability"] = 0.0

# Loop through each row of the schedule dataframe
for index, row in schedule.iterrows():
    # Extract the necessary information from the row
    team1 = row['Team 1']
    team2 = row['Team 2']
    espn_odds = row['ODDS BY']

    # Check if espn_odds is NaN before proceeding
    if pd.isna(espn_odds):
        continue  # Skip to the next iteration if odds are missing

    # Call the calculate_odds function with the relevant data
    [t1_win, t2_win, draw] = calculate_odds(team1, team2, espn_odds)

    # Add the returned probabilities to the schedule dataframe
    schedule.at[index, 'Team 1 Win Probability'] = t1_win
    schedule.at[index, 'Team 2 Win Probability'] = t2_win
    schedule.at[index, 'Draw Probability'] = draw

schedule.dropna(inplace=True)

# Display the updated schedule dataframe
display(schedule)


,Team 1,Team 2,Location,ODDS BY,Team 1 Win Probability,Team 2 Win Probability,Draw Probability
1,Leganés,Valencia,"Butarque, Leganés, Madrid, Spain",Line: LEG +175O/U: 1.5,0.341284,0.426273,0.232443
2,Espanyol,Mallorca,"Stage Front Stadium, Barcelona, Spain",Line: MLL +150O/U: 1.5,0.275717,0.522744,0.201539
3,Getafe,Osasuna,"Estadio Coliseum, Getafe, Spain",Line: GET +120O/U: 1.5,0.233280,0.542773,0.223946
4,Las Palmas,Celta Vigo,"Estadio Gran Canaria, Las Palmas de Gran Canar...",Line: CEL +125O/U: 2.5,0.209790,0.605668,0.184542
5,Real Valladolid,Rayo Vallecano,"José Zorrilla, Valladolid, Spain",Line: RAY +150O/U: 1.5,0.272204,0.501028,0.226768
6,Real Madrid,Villarreal,"Santiago Bernabéu, Madrid, Spain",Line: RMA -380O/U: 3.5,0.337559,0.456825,0.205617
7,Girona,Athletic Club,"Municipal de Montilivi, Girona, Spain",Line: GIR +135O/U: 2.5,0.275644,0.528180,0.196176
8,Alavés,Barcelona,"Mendizorroza, Vitoria-Gasteiz, Spain",Line: BAR -180O/U: 2.5,0.221314,0.665839,0.112848
9,Sevilla,Real Betis,"Ramón Sánchez-Pizjuán, Sevilla, Spain",Line: SEV +145O/U: 2.5,0.244880,0.520485,0.234635
10,Real Sociedad,Atlético Madrid,"Reale Arena, San Sebastián, Spain",Line: ATM +125O/U: 2.5,0.226248,0.565701,0.208051
